# Part A

- **Love**: Follow and maintain optimal distance
- **Fear**: Retreat from detected humans
- **Aggressive**: Relentless pursuit
- **Curious**: Cautious investigation with circling

Keyboard controls:
- 'l' = Love behavior
- 'f' = Fear behavior  
- 'a' = Aggressive behavior
- 'c' = Curious behavior
- 's' = Stop

**Step 1: Display Widgets**

Standard widget setup for displaying color and depth images side by side.

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create two widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# convert numpy array to jpeg for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

**Step 2: Load YOLO Model**

load the TensorRT YOLO model for human detection.

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11l_half.engine")

**Step 3: Camera System Initialization**

In [ ]:
# ZED camera initialization
import traitlets
import numpy as np
import pyzed.sl as sl
import threading
import motors
from traitlets.config.configurable import SingletonConfigurable

# initialize robot motor control
robot = motors.MotorsYukon(mecanum=False)

# Camera class with traitlets
class Camera(SingletonConfigurable):
    color_value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()

        # ZED camera initialization
        self.zed = sl.Camera()
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA  # 672x376
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Distance in mm

        # Open camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        self.runtime = sl.RuntimeParameters()
        self.thread_runnning_flag = False

        # Get camera resolution
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

    # threaded frame capture
    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                # Retrieve images
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
                
                # Convert BGRA to BGR 
                self.color_value_BGRA = self.image.get_data()
                self.color_value = cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                # Get depth as numpy array
                self.depth_image = np.asanyarray(self.depth.get_data())
                
    # thread control methods
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

# Initialize and start camera
camera = Camera()
camera.start()

**Step 4: Behavior Functions**

In [ ]:
# Global variable for current behavior mode
current_behavior = 'stop'

def detect_human(frame):
    """
    Human detection and distance calculation
    
    From Tutorial 5:
    - YOLO detection for humans (class 0)
    - Bounding box extraction: result.boxes.xyxy[i]
    - Center calculation: (bbox[0] + bbox[2]) / 2 for x, (bbox[1] + bbox[3]) / 2 for y
    - Distance extraction: camera.depth_image[center_y, center_x]

    """

    results = model(frame, verbose=False)
    
    # confidence threshold for detections
    conf_threshold = 0.5
    human_detected = False
    min_distance = float('inf')
    closest_bbox = None
    
    # iterate through detections
    for result in results:
        for i in range(len(result.boxes.cls)):
            if result.boxes.cls[i] == 0:
                # Check confidence (Tutorial 5)
                if result.boxes.conf[i] > conf_threshold:
                    human_detected = True
                    # Get bounding box coordinates (Tutorial 5)
                    bbox = result.boxes.xyxy[i]
                    
                    # Calculate center of bounding box (Tutorial 5 method)
                    # bbox format: [x1, y1, x2, y2]
                    center_x = int((bbox[0] + bbox[2]) / 2)
                    center_y = int((bbox[1] + bbox[3]) / 2)
                    
                    # Get depth at human center (Tutorial 5)
                    # Depth image indexing: depth_image[row, col] or depth_image[y, x]
                    # Returns distance in millimeters
                    if (0 <= center_y < camera.height and 0 <= center_x < camera.width):
                        distance = camera.depth_image[center_y, center_x]
                        
                        # NaN means no depth data, negative or zero means invalid
                        if not np.isnan(distance) and distance > 0:
                            # Track closest human
                            if distance < min_distance:
                                min_distance = distance
                                closest_bbox = bbox
    
    return human_detected, min_distance, closest_bbox

def love_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Love behavior - follow human and maintain comfortable distance
    
    Distance thresholds chosen based on testing:
    - < 450mm: Too close, back up
    - 450-600mm: Optimal distance, stop
    - 600-1200mm: Medium range, approach slowly
    - > 1200mm: Far away, approach faster
    
    """
    if human_detected and distance != float('inf'):
        # Draw detection box (Tutorial 5 pattern)
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 105, 180), 2)
        
        # Calculate if human is left/right of center (Tutorial 5 approach)
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Distance-based behavior (new thresholds for love behavior)
        if distance < 450:
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'LOVE: Too close {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        elif distance >= 450 and distance <= 600:
            robot.stop()
            cv2.putText(depth_colormap, f'LOVE: Perfect {distance:.0f}mm <3', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
        elif distance > 600 and distance <= 1200:
            # Medium distance - turn to center, then approach
            # 50 pixel threshold to avoid constant adjustment
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3) 
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3) 
            else:
                robot.forward(0.3) 
            cv2.putText(depth_colormap, f'LOVE: Approaching {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 180, 0), 2)
        else:  # distance > 1200
            # Far away - approach faster with turning
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.4)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.4)
            else:
                robot.forward(0.5)
            cv2.putText(depth_colormap, f'LOVE: Coming {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 100, 0), 2)
    else:
        # No human detected - stop and wait
        robot.stop()
        cv2.putText(depth_colormap, 'LOVE: Waiting...', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

def fear_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Fear behavior - retreat from humans
    
    Distance thresholds:
    - < 500mm: DANGER - fast retreat (0.6 speed)
    - < 900mm: CAUTION - medium retreat (0.4 speed)
    - < 1300mm: ALERT - slow retreat (0.2 speed)
    - >= 1300mm: SAFE - stop retreating
    """

    if human_detected and distance != float('inf'):
        # Draw red warning box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 0, 255), 2)
        
        # Graduated fear response
        if distance < 500:
            # Very close - fast retreat
            cv2.putText(depth_colormap, f'FEAR: DANGER {distance:.0f}mm!', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        elif distance < 900:
            # Medium distance - medium retreat
            robot.backward(0.4)
            cv2.putText(depth_colormap, f'FEAR: Caution {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 165, 255), 2)
        elif distance < 1300:
            # Far but cautious - slow retreat
            robot.backward(0.2)
            cv2.putText(depth_colormap, f'FEAR: Alert {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        else:
            # Safe distance - stop
            robot.stop()
            cv2.putText(depth_colormap, f'FEAR: Safe {distance:.0f}mm', 
                       (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    else:
        # No threat detected
        robot.stop()
        cv2.putText(depth_colormap, 'FEAR: No threat', 
                   (200, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

def aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Aggressive behavior - relentless pursuit and ramming
    
    - NEVER stops when target detected (shows more aggression)
    - Speed scales with distance (closer = more intense)
    - Continuously rams even at close range
    
    Speed scaling:
    - > 1000mm: 0.4 speed (chasing)
    - > 600mm: 0.6 speed (rushing)
    - > 300mm: 0.8 speed (engaging)
    - <= 300mm: 1.0 speed (destroy - keep ramming)
    
    """
    if human_detected and distance != float('inf'):
        # Draw orange attack indicator
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 165, 255), 2)
        
        # Calculate human position for tracking
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        # Intensity-based speed scaling
        if distance > 1000:
            speed = 0.4
            intensity = "chasing"
            color = (255, 165, 0)  # orange
        elif distance > 600:
            speed = 0.6
            intensity = "rushing"
            color = (255, 69, 0)  # red-orange
        elif distance > 300:
            speed = 0.8
            intensity = "engage"
            color = (255, 0, 0)  # red
        else:
            # Keep ramming at full speed
            speed = 1.0
            intensity = "RAMMING"
            color = (139, 0, 0)  # dark red
        
        # ALWAYS move forward or turn
        if bbox_center_x < frame_center_x - 50:
            robot.left(0.5)  # Turn to track
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} LEFT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        elif bbox_center_x > frame_center_x + 50:
            robot.right(0.5)  # Turn to track
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} RIGHT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        else:
            robot.forward(speed)  # Charge forward
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} {distance:.0f}mm!', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        print(f'Aggression: {intensity} {distance:.0f}mm (speed: {speed})', end='\r')
    else:
        robot.spinRight(0.3)  # Spin to search
        cv2.putText(depth_colormap, 'AGGRESSIVE: Searching...', 
                   (150, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        print('Aggression: Checking for target', end='\r')

def curious_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """
    Curious behavior - cautious investigation with circling
    
    Behavior pattern:
    - Far (>800mm): Approach with tracking
    - Medium (400-800mm): Slow approach
    - Close (<400mm): Circle around to investigate
    - None detected: Explore environment
    
    Circling achieved by combining forward movement with rotation
    """

    if human_detected and distance != float('inf'):
        # Draw cyan curiosity indicator
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (255, 255, 0), 2)
        
        # Calculate position for tracking
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2
        
        if distance < 400:
            # Close range - circle around to investigate
            robot.forward(0.2)  # Move forward slowly
            robot.right(0.3)    # While turning
            cv2.putText(depth_colormap, f'CURIOUS: Investigating {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        elif distance < 800:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.2)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.2)
            else:
                robot.forward(0.2)
            cv2.putText(depth_colormap, f'CURIOUS: Approaching {distance:.0f}mm', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 100), 2)
        else:
            if bbox_center_x < frame_center_x - 50:
                robot.left(0.3)
            elif bbox_center_x > frame_center_x + 50:
                robot.right(0.3)
            else:
                robot.forward(0.4)
            cv2.putText(depth_colormap, f'CURIOUS: Interested {distance:.0f}mm', 
                       (140, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 200, 0), 2)
    else:
        # No human detected - explore environment
        robot.forward(0.2)
        robot.right(0.2)
        cv2.putText(depth_colormap, 'CURIOUS: Exploring...', 
                   (180, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)

**Step 5: Keyboard Control and Main Callback**

Based on Answer_keyboard_control.ipynb pattern for keyboard input handling.

In [ ]:
text_input = widgets.Text(
    value='',
    placeholder='Type: l=Love, f=Fear, a=Aggressive, c=Curious, s=Stop',
    description='Behavior:',
    disabled=False
)

# observe text changes
def on_text_change(change):
    """
    Handle keyboard input to switch behaviors
    """
    global current_behavior
    input_value = change['new']
    
    if len(input_value) > 0:
        last_char = input_value[-1].lower()
        
        # Switch behavior based on input
        if last_char == 'l':
            current_behavior = 'love'
            print('\nSwitched to LOVE behavior')
        elif last_char == 'f':
            current_behavior = 'fear'
            print('\nSwitched to FEAR behavior')
        elif last_char == 'a':
            current_behavior = 'aggressive'
            print('\nSwitched to AGGRESSIVE behavior')
        elif last_char == 'c':
            current_behavior = 'curious'
            print('\nSwitched to CURIOUS behavior')
        elif last_char == 's':
            current_behavior = 'stop'
            robot.stop()
            print('\nSTOPPED - Robot halted')

text_input.observe(on_text_change, names='value')
display(text_input)

def main_callback(change):
    """
    Main callback function - executes current behavior    
    """
    # Get updated frame
    frame = change['new']
    
    # Create depth colormap for display
    # Converts depth values to color visualization
    depth_colormap = cv2.applyColorMap(
        cv2.convertScaleAbs(camera.depth_image, alpha=0.03), 
        cv2.COLORMAP_JET)
    
    # Detect humans and measure distance (Tutorial 5 method)
    human_detected, distance, bbox = detect_human(frame)
    
    # Execute behavior based on current mode
    if current_behavior == 'love':
        love_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'fear':
        fear_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'aggressive':
        aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox)
    elif current_behavior == 'curious':
        curious_behavior(frame, depth_colormap, human_detected, distance, bbox)
    else:  # stop
        robot.stop()
        cv2.putText(depth_colormap, 'STOPPED', 
                   (250, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (128, 128, 128), 2)
    
    # Display current behavior mode on frame
    cv2.putText(frame, f'Mode: {current_behavior.upper()}', 
               (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Display images
    scale = 0.3
    resized_color = cv2.resize(frame, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                               interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_color)
    display_depth.value = bgr8_to_jpeg(resized_depth)

camera.observe(main_callback, names=['color_value'])

print('\n=== Braitenberg Behaviors Active ===')
print('Press keys to switch behaviors:')
print('  l = Love (follow human)')
print('  f = Fear (retreat from human)')
print('  a = Aggressive (charge at human)')
print('  c = Curious (investigate human)')
print('  s = Stop (halt robot)')
print('===================================')

**To stop the system:**

In [ ]:
# stop camera thread and motors
camera.stop()
print('System stopped - camera and robot halted')

## Implementation Notes

### Distance Calculation
Distance is measured by:
1. Getting bounding box from YOLO: `bbox = result.boxes.xyxy[i]` (format: [x1, y1, x2, y2])
2. Calculating center: `center_x = (bbox[0] + bbox[2]) / 2`, `center_y = (bbox[1] + bbox[3]) / 2`
3. Reading depth at center: `distance = camera.depth_image[center_y, center_x]`
4. Depth image returns distance in millimeters


### Behavior Distance Thresholds:

**Love:** 450mm (too close) | 450-600mm (perfect) | 600-1200mm (medium) | >1200mm (far)

**Fear:** 500mm (danger) | 900mm (caution) | 1300mm (alert) | >1300mm (safe)

**Aggressive:** 300mm (ram) | 600mm (engage) | 1000mm (rush) | >1000mm (chase)

**Curious:** 400mm (investigate) | 800mm (approach) | >800mm (interested)